In [66]:
import glob
from pdf2image import convert_from_path
import pandas as pd
import requests
from io import StringIO

train_pdf = glob.glob('./标书实体抽取挑战赛公开数据/train/pdf/*')
train_label = glob.glob('./标书实体抽取挑战赛公开数据/train/label/*')

train_pdf.sort()
train_label.sort()

test_pdf = glob.glob('./标书实体抽取挑战赛公开数据/test/pdf/*')
test_label = glob.glob('./标书实体抽取挑战赛测试集要素名称/*')

test_pdf.sort()
test_label.sort()

from paddleocr import PaddleOCR, draw_ocr
ocr = PaddleOCR(use_angle_cls=True, lang="ch")

[2023/08/30 14:47:19] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/lyz/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/lyz/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dic

In [100]:
for pdf_path, label_path in zip(test_pdf[3:], test_label[3:]):
    
    # 步骤1：读取待提取的信息
    df = pd.read_excel(label_path)
    df['要素点'] = ''
    
    # 步骤2：提取pdf内容
    pages = convert_from_path(pdf_path, 200)
    pages = [pages[0], pages[1], pages[-1]]
    content = ''
    for count, page in enumerate(pages):
        page.save(f'out.jpg', 'JPEG')

        result = ocr.ocr('out.jpg', cls=True)
        for idx in range(len(result)):
            res = result[idx]
            for line in res:
                content += line[1][0] + '\n'

        content += '\n'
    
    # 步骤3：将pdf内容分页提取
    url = "https://oa.api2d.site/v1/chat/completions"
    headers = {
        'Content-Type': 'application/json',    
        'Authorization': 'Bearer 替换为你的api-key',
    }

    table_extract_result = []
    total_page = len(content.split('\n'))
    for idx in range(total_page // 50 + 1):
        input_content = '\n'.join(content.split('\n')[idx*50:(idx+1)*50])

        data = {
            "model": "gpt-3.5-turbo",

            "messages": [{"role": "user", "content": 
            f"""
            【任务】帮我从内容中提取信息，填充下面的表格，如果无法填充，则填无。只需要输出待填充表格。

            【输入信息】
            {input_content}

            【待填充表格内容】
            {df.to_markdown()}
            """}]
        }

        response = requests.post(url, headers=headers, json=data)
        table_extract_result.append(response.json()['choices'][0]['message']['content'])
    
    # 步骤4：将最终结果汇总为最终格式
    table_extract_result = '\n\n'.join(table_extract_result)

    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": 
        f"""
        【任务】帮将表格信息进行合并，只需要输出待填充表格。

        【输入信息】
        {table_extract_result}

        【表格最终格式】
        {df.to_markdown()}
        """}]
    }

    response = requests.post(url, headers=headers, json=data)
    
    print(pdf_path)
    print(response.json()['choices'][0]['message']['content'])
    
    try:
        pd.read_csv(
            StringIO(response.json()['choices'][0]['message']['content'].replace(' ', '')),  # Get rid of whitespaces
            sep='|',
            index_col=1
        ).dropna(
            axis=1,
            how='all'
        ).iloc[1:]['要素点'].to_csv('./submit/' + pdf_path.split('/')[-1][:-4] + '.csv', index=None, header=None)
    except:
        pass

[2023/08/30 15:46:16] ppocr DEBUG: dt_boxes num : 10, elapse : 0.5190646648406982
[2023/08/30 15:46:16] ppocr DEBUG: cls num  : 10, elapse : 0.05770516395568848
[2023/08/30 15:46:17] ppocr DEBUG: rec_res num  : 10, elapse : 1.3793835639953613
[2023/08/30 15:46:18] ppocr DEBUG: dt_boxes num : 16, elapse : 0.4673466682434082
[2023/08/30 15:46:18] ppocr DEBUG: cls num  : 16, elapse : 0.08149027824401855
[2023/08/30 15:46:19] ppocr DEBUG: rec_res num  : 16, elapse : 1.5296516418457031
[2023/08/30 15:46:20] ppocr DEBUG: dt_boxes num : 36, elapse : 0.49208545684814453
[2023/08/30 15:46:20] ppocr DEBUG: cls num  : 36, elapse : 0.18323349952697754
[2023/08/30 15:46:24] ppocr DEBUG: rec_res num  : 36, elapse : 3.9673149585723877
./标书实体抽取挑战赛公开数据/test/pdf/CT201011166974-信科集团-支出类-物资购销合同-40页扫描件.pdf
|    | 要素名称                     | 要素点   |
|---:|:-----------------------------|:---------|
|  0 | 发票税率                     |          |
|  1 | 负责人签名                   |          |
|  2 | 合同尾部_受托方_受托方名称值 

In [101]:
!\rm -rf ./submit/.ipynb_checkpoints/

In [102]:
!zip -r submit.zip submit

  adding: submit/ (stored 0%)
  adding: submit/CT201710984626-中交建筑-收入类-内部任务书-4页扫描件.csv (deflated 74%)
  adding: submit/CT201411033436-信科集团-支出类-设备 (含船机)购销合同-28页扫描件.csv (deflated 71%)
  adding: submit/CT201111215740-民航机场建设集团-支出类-设计合同-8页扫描件.csv (deflated 23%)
  adding: submit/CT202110629429-四航局-收入类-赔偿、补偿类合同-3页扫描件.csv (deflated 67%)
  adding: submit/CT202211297356-一航局-收入类-设计施工总承包合同-34页扫描件.csv (stored 0%)
  adding: submit/CT202311405769-二公院-收入类-专业分包合同-106页扫描件.csv (deflated 74%)
  adding: submit/CT201310863030-民航机场建设集团-支出类-专业分包合同-23页扫描件.csv (deflated 67%)
  adding: submit/CT201810217655-一航局-收入类-专业分包合同-68页扫描件.csv (stored 0%)
  adding: submit/CT202110330559-三航局-收入类-赔偿、补偿类合同-3页扫描件.csv (deflated 71%)
  adding: submit/CT201311324586-中交建筑-支出类-劳务分包合同-33页扫描件.csv (deflated 22%)
  adding: submit/CT201210918007-一公局-支出类-专业分包合同-26页扫描件.csv (deflated 53%)
  adding: submit/CT202311466757-中国城乡-收入类-其他工程类合同-7页扫描件.csv (stored 0%)
  adding: submit/CT202211460211-四航局-收入类-设计施工采购总承包合同-27页扫描件.csv (deflated 15%)
  ad